In [1]:
# load binary file
import numpy as np
import time
from suite2p.run_s2p import run_s2p
import imp

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'diameter': 12,
    'tau': 2.,
    'fs': 2.5,
    'data_path': ['C:/Drive/suite2python/data1/'],
    #'data_path': 'C:\Drive\suite2python\data',
    'subfolders': ['2', '5', '6'],
    'num_workers': 0,
    'reg_tif':False,
    'max_iterations':10,
      }


# copy tiff to a binary
i0 = tic()
ops1 = run_s2p(ops)
print(toc(i0))
#ops1 = register.tiff_to_binary(ops)

found ops1 and pre-registered binaries
overwriting ops1 with new ops
skipping registration...
time 35.7089. SVD computed
cells: 224, cost: 0.6161, time: 20.3722
cells: 403, cost: 0.5994, time: 44.1679
cells: 522, cost: 0.5912, time: 69.5143
cells: 577, cost: 0.5881, time: 96.7187


KeyboardInterrupt: 

time 340.7931. SVD computed
cells: 224, cost: 0.6161, time: 22.1188
